In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Añadimos la carpeta "utils" al path para poder importar nuestros módulos propios
sys.path.append("../utils")

import toolbox_ML as tb
import bootcampviztools as bt

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [2]:
df = pd.read_csv("../data_sample/survey.csv")
df.columns = df.columns.str.lower()
display(df)

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,No,Yes,No,No,No,No,Don't know,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,Yes,Yes,Yes,Yes,No,No,Yes,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,No,Yes,Yes,Yes,No,No,Don't know,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,Yes,Yes,No,Yes,No,No,Don't know,Don't know,Yes,No,No,No,No,No,No,No,NaN


In [5]:
df.country.value_counts()

country
United States             751
United Kingdom            185
Canada                     72
Germany                    45
Netherlands                27
Ireland                    27
Australia                  21
France                     13
India                      10
New Zealand                 8
Poland                      7
Italy                       7
Sweden                      7
Switzerland                 7
South Africa                6
Brazil                      6
Belgium                     6
Israel                      5
Singapore                   4
Bulgaria                    4
Russia                      3
Austria                     3
Finland                     3
Mexico                      3
Denmark                     2
Greece                      2
Portugal                    2
Colombia                    2
Croatia                     2
Slovenia                    1
Costa Rica                  1
Latvia                      1
Uruguay                     1
Sp

In [6]:
# Guardar la original por si queremos probar luego
df["country_raw"] = df["country"]

# Guardamos los 10 países con más encuestados
top_10_paises = [
    "United States", "United Kingdom", "Canada", "Germany", "Netherlands",
    "Ireland", "Australia", "France", "India", "New Zealand"
]

# Creamos la nueva columna
df["country_grouped"] = df["country"].apply(lambda x: x if x in top_10_paises else "Other")

In [8]:
df["country_grouped"].value_counts()

country_grouped
United States     751
United Kingdom    185
Other             100
Canada             72
Germany            45
Ireland            27
Netherlands        27
Australia          21
France             13
India              10
New Zealand         8
Name: count, dtype: int64

In [10]:
df.gender.value_counts()

gender
Male                                              615
male                                              206
Female                                            121
M                                                 116
female                                             62
F                                                  38
m                                                  34
f                                                  15
Make                                                4
Male                                                3
Woman                                               3
Cis Male                                            2
Man                                                 2
Female                                              2
Female (trans)                                      2
Male-ish                                            1
Trans-female                                        1
Male (CIS)                                          1
Mal                  

In [13]:
# Limpiamos y transformamos todas las respuestas para que funcione bien nuestro clasificador
df["gender"] = df["gender"].str.lower().str.strip()

# Clasificador ayudándonos del value counts de gender
def clasificar_genero(genero):
    if genero in ["male", "m", "man", "cis male", "msle", "mal", "mail", "maile", "make", 
               "guy (-ish) ^_^", "male (cis)", "malr", "cis man"]:
        return "male"
    elif genero in ["female", "f", "woman", "cis female", "femake", "femail", "cis-female/femme", 
               "female (cis)"]:
        return "female"
    else:
        return "other"

df["gender"] = df["gender"].apply(clasificar_genero)

# Comprobamos que está correcto
print("Distribución final de género (en solo tres categorías):")
print(df["gender"].value_counts())

Distribución final de género (en solo tres categorías):
gender
male      991
female    247
other      21
Name: count, dtype: int64


In [15]:
tb.describe_df(df)

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
DATA_TYPE,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
MISSINGS (%),0.0,0.0,0.0,0.0,40.905481,1.429706,0.0,0.0,20.969023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.973789
UNIQUE_VALUES,1246,53,3,48,45,2,2,2,4,6,2,2,3,3,3,3,3,5,3,3,3,3,3,3,3,2,160
CARDIN (%),98.97,4.21,0.24,3.81,3.57,0.16,0.16,0.16,0.32,0.48,0.16,0.16,0.24,0.24,0.24,0.24,0.24,0.4,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.16,12.71


In [17]:
df.age.value_counts()

age
 29             85
 32             82
 26             75
 27             71
 33             70
 28             68
 31             67
 34             65
 30             63
 25             61
 35             55
 23             51
 24             46
 37             43
 38             39
 36             37
 39             33
 40             33
 43             28
 41             21
 22             21
 42             20
 21             16
 45             12
 46             12
 44             11
 19              9
 18              7
 50              6
 48              6
 20              6
 51              5
 56              4
 49              4
 57              3
 55              3
 54              3
 47              2
 60              2
-29              1
 329             1
 99999999999     1
 58              1
 62              1
 65              1
-1726            1
 5               1
 53              1
 61              1
 8               1
 11              1
-1               1
 72     

In [19]:
# Filtramos edades claramente erróneas
df = df[(df["age"] >= 15) & (df["age"] <= 80)]

# Comprobamos
print("Distribución de edad tras limpieza:")
print(df["age"].describe())

Distribución de edad tras limpieza:
count    1251.000000
mean       32.076739
std         7.288272
min        18.000000
25%        27.000000
50%        31.000000
75%        36.000000
max        72.000000
Name: age, dtype: float64


In [21]:
orden_empleados = {
    "1-5": 1,
    "6-25": 2,
    "26-100": 3,
    "100-500": 4,
    "500-1000": 5,
    "More than 1000": 6
}
df["no_employees_encoded"] = df["no_employees"].map(orden_empleados)
df.no_employees_encoded.value_counts(dropna=False)

C:\Users\dchev\AppData\Local\Temp\ipykernel_728\1000364745.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["no_employees_encoded"] = df["no_employees"].map(orden_empleados)


no_employees_encoded
2    289
3    288
6    281
4    175
1    158
5     60
Name: count, dtype: int64